In [ ]:

# =========================
# Task 1 - KNN Classifier from Scratch + Bonus Decision Boundary
# =========================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# ----------------------
# 1. Load & Preprocess Data
# ----------------------
df = pd.read_csv("data.csv")  # Ensure file is uploaded in Colab

# Drop unnecessary columns
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

# Encode target
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

# Separate features & labels
X = df.drop(columns=['diagnosis']).values
y = df['diagnosis'].values

# Normalize features
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

# Train-test split
np.random.seed(42)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
split = int(0.8 * len(indices))
train_idx, test_idx = indices[:split], indices[split:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# ----------------------
# 2. Distance Functions
# ----------------------
def euclidean(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

def manhattan(a, b):
    return np.sum(np.abs(a - b))

def minkowski(a, b, p=3):
    return np.sum(np.abs(a - b) ** p) ** (1 / p)

def cosine_similarity(a, b):
    return 1 - (np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

def hamming_distance(a, b):
    return np.mean(a != b)

distance_funcs = {
    "Euclidean": euclidean,
    "Manhattan": manhattan,
    "Minkowski": lambda a, b: minkowski(a, b, p=3),
    "Cosine": cosine_similarity,
    "Hamming": hamming_distance
}

# ----------------------
# 3. KNN Implementation
# ----------------------
def knn_predict(X_train, y_train, x_test, k, dist_func):
    distances = [dist_func(x_test, x_train) for x_train in X_train]
    k_indices = np.argsort(distances)[:k]
    k_labels = [y_train[i] for i in k_indices]
    return Counter(k_labels).most_common(1)[0][0]

def knn_evaluate(X_train, y_train, X_test, y_test, k, dist_func):
    y_pred = [knn_predict(X_train, y_train, x, k, dist_func) for x in X_test]
    accuracy = np.mean(y_pred == y_test)
    return accuracy, np.array(y_pred)

# ----------------------
# 4. Experiment
# ----------------------
K_values = [3, 4, 9, 20, 47]
results = {}

for dist_name, dist_func in distance_funcs.items():
    accuracies = []
    for k in K_values:
        acc, _ = knn_evaluate(X_train, y_train, X_test, y_test, k, dist_func)
        accuracies.append(acc)
    results[dist_name] = accuracies

# ----------------------
# 5. Best K & Distance
# ----------------------
best_acc = 0
best_k = None
best_dist = None
best_pred = None

for dist_name, dist_func in distance_funcs.items():
    for k in K_values:
        acc, y_pred = knn_evaluate(X_train, y_train, X_test, y_test, k, dist_func)
        if acc > best_acc:
            best_acc = acc
            best_k = k
            best_dist = dist_name
            best_pred = y_pred

# ----------------------
# 6. Confusion Matrix, Precision, Recall
# ----------------------
TP = np.sum((best_pred == 1) & (y_test == 1))
TN = np.sum((best_pred == 0) & (y_test == 0))
FP = np.sum((best_pred == 1) & (y_test == 0))
FN = np.sum((best_pred == 0) & (y_test == 1))

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

print("Best K:", best_k)
print("Best Distance Metric:", best_dist)
print("Best Accuracy:", round(best_acc * 100, 2), "%")
print("\nConfusion Matrix:")
print(f"TP = {TP}, TN = {TN}, FP = {FP}, FN = {FN}")
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))

# ----------------------
# 7. Plot Accuracy vs K
# ----------------------
plt.figure(figsize=(8, 6))
for dist_name, acc_list in results.items():
    plt.plot(K_values, acc_list, marker='o', label=dist_name)
plt.title("K values vs Accuracy for different Distance Metrics")
plt.xlabel("K")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# ----------------------
# 8. BONUS - Decision Boundary (PCA to 2D)
# ----------------------
from sklearn.decomposition import PCA

# Reduce to 2D
pca = PCA(n_components=2)
X_train_2D = pca.fit_transform(X_train)
X_test_2D = pca.transform(X_test)

# Create mesh grid
x_min, x_max = X_train_2D[:, 0].min() - 0.1, X_train_2D[:, 0].max() + 0.1
y_min, y_max = X_train_2D[:, 1].min() - 0.1, X_train_2D[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Predict over the grid
dist_func = distance_funcs[best_dist]
Z = np.array([knn_predict(X_train_2D, y_train, np.array([x, y]), best_k, dist_func)
              for x, y in np.c_[xx.ravel(), yy.ravel()]])
Z = Z.reshape(xx.shape)

# Plot decision boundary
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)
plt.scatter(X_train_2D[:, 0], X_train_2D[:, 1], c=y_train, cmap=plt.cm.coolwarm, edgecolor='k', s=20, label="Train")
plt.scatter(X_test_2D[:, 0], X_test_2D[:, 1], c=y_test, cmap=plt.cm.coolwarm, marker='x', s=40, label="Test")
plt.title(f"Decision Boundary (Best K={best_k}, {best_dist} Distance)")
plt.legend()
plt.show()



## Observations & Inference

1. **Best Model Parameters:**  
   - The highest test accuracy (**96.49%**) was achieved using **K = 3** and **Manhattan Distance**.  
   - This combination gave **Precision = 1.0** and **Recall ≈ 0.915**, meaning all predicted malignant cases were correct, but a few malignant cases were missed (false negatives).  

2. **Effect of K on Accuracy:**  
   - For smaller `K` values (3, 4), accuracy was generally higher.  
   - As `K` increased (especially at `K=47`), accuracy slightly decreased for most distance metrics due to excessive smoothing and reduced sensitivity to local patterns.  

3. **Effect of Distance Metrics:**  
   - **Manhattan Distance** performed best overall, closely followed by **Euclidean** and **Minkowski**.  
   - **Cosine Similarity** and **Hamming Distance** produced lower accuracies, suggesting that magnitude-based distances are more effective for this dataset than purely directional or binary comparison metrics.  

4. **Confusion Matrix Insights:**  
   - **True Positives (43)** and **True Negatives (67)** dominate, with **0 False Positives** (no benign tumor misclassified as malignant).  
   - **4 False Negatives** occurred, meaning a few malignant cases were incorrectly classified as benign. This is critical in medical contexts as it might delay treatment.  

5. **Decision Boundary Visualization:**  
   - After PCA reduction to 2D, the decision boundary shows a clear separation between malignant and benign samples for the chosen model, with only minor overlaps at the class boundary.  

**Final Inference:**  
A KNN classifier with **K=3** and **Manhattan Distance** offers the best trade-off between sensitivity and specificity for this dataset. However, in medical diagnosis scenarios, minimizing false negatives should be prioritized, so future work could explore weighted KNN or adjusting decision thresholds to improve recall without sacrificing too much precision.
